# MLflow Kserve integration test
- start experiment 
- train model
- save artifact
- create kserve inference service with the artifact
- make sure service is up
- run prediction

## Setup

In [1]:
# pin the mlflow client to match the version of the deployed MLflow server
!pip install minio mlflow==2.1.1 boto3 tenacity -q

### Import required packages

In [2]:
import mlflow
import numpy as np
import pandas as pd
import requests

from kubernetes import client as k8s_client, config as k8s_config
from mlflow.models.signature import infer_signature
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

from sklearn.model_selection import train_test_split
from tenacity import retry, stop_after_attempt, wait_exponential

## Download Data

In [3]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
data.shape

(1599, 12)

## Preprocess Data

In [5]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]


## Create MLflow experiment

In [6]:
wine_experiment_name = "My Wine Experiment Kserve"
experiment = mlflow.get_experiment_by_name(wine_experiment_name)
experiment_id = (
    mlflow.create_experiment(name=wine_experiment_name)
    if experiment is None
    else experiment.experiment_id
)

In [7]:
# check that the experiment was created successfully
assert mlflow.get_experiment(experiment_id).name == wine_experiment_name, f"Failed to create experiment {wine_experiment_name}!"

## Train and store model

In [8]:
def experiment(alpha, l1_ratio):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name='wine_models', experiment_id=experiment_id) as run:
            mlflow.set_tag("author", "kf-testing")
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            pred_y = lr.predict(test_x)
            mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
            mlflow.log_metric("r2", r2_score(test_y, pred_y))
            mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

            signature = infer_signature(test_x, pred_y)
            result = mlflow.sklearn.log_model(lr, "model", registered_model_name="wine-elasticnet", signature=signature)
            model_uri = f"{mlflow.get_artifact_uri()}/{result.artifact_path}"
    
    return run, model_uri

In [9]:
run, model_uri = experiment(0.5, 0.5)

2023/10/23 12:45:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Registered model 'wine-elasticnet' already exists. Creating a new version of this model...
2023/10/23 12:45:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: wine-elasticnet, version 3
Created version '3' of model 'wine-elasticnet'.


In [10]:
model_uri

's3://mlflow/1/d4087e8103db4f9ba82fd81ec164507e/artifacts/model'

## Deploy Kserve's InferenceService

In [11]:
def deploy(
    kserve_inference_name: str = "wine-regressor",
    model_uri: str = "default_model_uri",
):
    import yaml

    from kubernetes import client, config

    manifest = """
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: """+kserve_inference_name+"""
spec:
  predictor:
    serviceAccountName: kserve-controller-s3
    sklearn:
      storageUri: """+model_uri+"""
    """
    


    with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
        namespace = f.read().strip()

    config.load_incluster_config()
    api_instance = client.ApiClient()
    custom_api = client.CustomObjectsApi(api_instance)

    try:
        api_response = custom_api.create_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            plural="inferenceservices",
            namespace=namespace,
            body=yaml.safe_load(manifest),
        )
        print("Custom Resource applied successfully.")
        print(api_response)
    except client.rest.ApiException as e:
        print(f"Failed to apply Custom Resource: {e}")

In [12]:
KSERVE_INFERENCE_NAME = "wine-regressor"
deploy(kserve_inference_name=KSERVE_INFERENCE_NAME, model_uri=model_uri)

Custom Resource applied successfully.
{'apiVersion': 'serving.kserve.io/v1beta1', 'kind': 'InferenceService', 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'RawDeployment'}, 'creationTimestamp': '2023-10-23T12:45:54Z', 'generation': 1, 'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:predictor': {'.': {}, 'f:serviceAccountName': {}, 'f:sklearn': {'.': {}, 'f:storageUri': {}}}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2023-10-23T12:45:53Z'}], 'name': 'wine-regressor', 'namespace': 'user123', 'resourceVersion': '47115', 'uid': 'f40133b7-9b51-4e01-9d3d-ba534af93922'}, 'spec': {'predictor': {'model': {'modelFormat': {'name': 'sklearn'}, 'name': '', 'resources': {}, 'storageUri': 's3://mlflow/1/d4087e8103db4f9ba82fd81ec164507e/artifacts/model'}, 'serviceAccountName': 'kserve-controller-s3'}}}


### Wait for active InferenceService

In [13]:
with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
    NAMESPACE = f.read().strip()

k8s_config.load_incluster_config()
api_instance = k8s_client.ApiClient()
custom_api = k8s_client.CustomObjectsApi(api_instance)

In [14]:
def get_kserve_inference_service(name, namespace):
    """Get Kserve InferenceService by name."""
    return custom_api.get_namespaced_custom_object(
        group="serving.kserve.io",
        version="v1beta1",
        plural="inferenceservices",
        namespace=namespace,
        name=name,
    )

def delete_kserve_inference_service(name, namespace):
    """Delete Kserve InferenceService by name."""
    return custom_api.delete_namespaced_custom_object(
        group="serving.kserve.io",
        version="v1beta1",
        plural="inferenceservices",
        namespace=namespace,
        name=name,
    )

In [15]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(50),
    reraise=True,
)
def assert_kserve_deployment_available(kserve_inference_name, namespace):
    """Wait for the SeldonDeployment to become available."""
    try:
        sd = get_kserve_inference_service(kserve_inference_name, namespace)
    except ApiException as err:
        assert err.status != 404, f"Kserve InferenceService {kserve_inference_name} not found!"
        raise
    status = sd.get("status")
    assert status is not None, "Kserve InferenceService status not yet available!"
    assert status.get("conditions")[-1].get("status") == "True"

In [16]:
assert_kserve_deployment_available(KSERVE_INFERENCE_NAME, NAMESPACE)

### Run Prediction

In [17]:
sd = get_kserve_inference_service(KSERVE_INFERENCE_NAME, NAMESPACE)
url = sd['status']['address']['url']
print("Kserve InferenceService URL:", url)

Kserve InferenceService URL: http://wine-regressor.user123.svc.cluster.local


In [18]:
inference_input = {
    "instances": [
        [10.1, 0.37, 0.34, 2.4, 0.085, 5.0, 17.0, 0.99683, 3.17, 0.65, 10.6]
    ]
}
response = requests.post(f"http://{KSERVE_INFERENCE_NAME}-predictor-default.user123.svc.cluster.local/v1/models/{KSERVE_INFERENCE_NAME}:predict", json=inference_input)
print(response.text)

{"predictions":[5.743164652981532]}


In [19]:
res = response.json()
# verify that the predictions are as expected
assert res.get("predictions"), "Failed to get predictions!"
predictions = res["predictions"]
assert len(predictions) == 1, "Predictions not in the expected format!"

In [ ]:
# 5.737976780306664